In [1]:
import pandas as pd
import xgboost as xgb
import os

import dtale

import scipy.stats as stats
from config.envsConfig import ROOT_DIR
import numpy as np

import psycopg2

####### read files: #########
def readDF(fileDir):
    featureDF = pd.read_csv(fileDir)
    return featureDF

### descrição do problema:

### Ja ta com as features
### Ja ta tudo na mesma tabela
### Problema de classificacao

###

DATABASE = os.environ.get("DATABASE")
USER = os.environ.get("USER")
PASSWORD = os.environ.get("PASSWORD")
HOST = os.environ.get("HOST")
PORT = os.environ.get("PORT")

filePathData = os.path.join(ROOT_DIR,"data")
filePathSQL = os.path.join(ROOT_DIR,"data","tmp")

fileDir = os.path.join(ROOT_DIR,"data","tabular-playground-series-may-2022", "train.csv")

idxName = ['id']
#catFList = ['f_27']
catFList = ['f_27',"f_07","f_08","f_09","f_10","f_11","f_12"]
#discFList = ["f_07","f_08","f_09","f_10","f_11","f_12","f_13","f_14","f_15","f_16","f_17","f_18","f_29","f_30"]
discFList = ["f_13","f_14","f_15","f_16","f_17","f_18","f_29","f_30"]
tgtFList = ["target"]

pandasSQLTypes={"int64":"bigint","object":"varchar(100)","float64":"double precision"}

In [2]:
featureDF = readDF(fileDir)
featureDF.set_index(idxName[0],inplace=True,drop=False)
numFList = list(set(featureDF.columns) - set(tgtFList) - set(discFList) - set(catFList))
numFList.sort()

numDF = featureDF
#numDF = numDF.sample(n=50000)

numDFMelt = pd.melt(numDF, id_vars = idxName + catFList, value_vars=numFList+discFList,value_name='numFeatures')

dim_fDF = pd.DataFrame(list(featureDF.columns),columns=["variable"])
dim_fDF.drop_duplicates(inplace=True)
dim_idDF = pd.DataFrame(list(featureDF.index),columns=idxName)
dim_idDF.drop_duplicates(inplace=True)

In [3]:
numDFMelt.head(5)

,id,f_27,f_07,f_08,f_09,f_10,f_11,f_12,variable,numFeatures
0,0,ABABDADBAB,1,5,1,3,3,3,f_00,-1.373246
1,1,ACACCADCEB,1,3,4,0,2,3,f_00,1.697021
2,2,AAAEABCKAD,1,0,2,6,6,4,f_00,1.681726
3,3,BDBBAACBCB,3,2,1,0,1,6,f_00,-0.118172
4,4,BDBCBBCHFE,3,3,0,4,3,0,f_00,1.148481


In [4]:
colunas = list(numDFMelt['variable'].drop_duplicates())

dfList = []
binListNp = []
binListTmp = []
binList = []
for col in colunas:

    numDFMelt_1 = (numDFMelt[numDFMelt['variable'] == col]).copy()
    #bins = pd.cut(numDFMelt_1['numFeatures'], 3)

    binQuant = int(round((len(numDFMelt_1))**(1/2),0))

    binList = pd.cut(numDFMelt_1['numFeatures'], binQuant)

    #nao funciona para as slices?

    numDFMelt_1['ranking'] = (numDFMelt_1['numFeatures'].rank(axis=0,method='max',ascending=True)-0.05) / len(numDFMelt_1)
    numDFMelt_1['qqplot'] = stats.norm.ppf(numDFMelt_1['ranking'])

    for rows in binList:
        binListTmp.append([col,rows.left])

    binListNp = np.array(binListTmp)
    binListNp = np.append(binListNp, np.array([numDFMelt_1['id'].to_numpy()]).T, axis=1)
    binListNp = np.append(binListNp, np.array([numDFMelt_1['qqplot'].to_numpy()]).T, axis=1)

    dfList.append(pd.DataFrame(binListNp,columns=['variable','bin','id','qqplot']))

    binListTmp = []
    binListNp = []

binDf = pd.concat(dfList)
binDf['qqplot'] = binDf['qqplot'].astype('float64')
binDf['bin'] = binDf['bin'].astype('float64')
binDf['id'] = binDf['id'].astype('int64')

##slow
numDFMelt = numDFMelt.merge(binDf, how='left',on=["id","variable"])

In [5]:
#numDFx = numDF.drop(columns=['id'])
#d = dtale.show(numDFx)
#print(d._url)
#d.open_browser()

In [6]:
def createCorrelDFrame(df):

    correl = df.corr()
    correl = pd.melt(correl, value_vars=numFList+discFList+tgtFList,value_name='correl', ignore_index=False)
    correl.reset_index(inplace=True)
    correl = correl.rename(columns={'index':'variable1','variable':'variable2'})
    
    return correl

def createColumnStr(df,typeMap=pandasSQLTypes):

    dfcolumns = df.columns
    dfTypes = df.dtypes

    strList = ""

    for idx in range(0,len(dfcolumns)):

        if idx == len(dfcolumns)-1 and idx == 0:
            strList = "(" + strList + " " + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ")"
        elif idx == len(dfcolumns)-1:
            strList = strList + " " + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ")"
        elif idx == 0:
            strList = "(" + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ","
        else:
            strList = strList + " " + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ","

    return strList

############
def copyDataToSQL(df,tablename,path = filePathSQL):

    pg_conn  = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)
    cur = pg_conn.cursor()

    filePath = os.path.join(path,tablename + ".csv")

    df.to_csv(filePath,index=False, header=False)

    strList = createColumnStr(df)

    sqlCreate = '''CREATE TABLE IF NOT EXISTS ''' + tablename + ''' ''' + strList
    print(sqlCreate)

    sqlCopy = '''COPY ''' + tablename + ''' FROM ''' + "'"+filePath+"'" + ''' DELIMITER ',' CSV; '''
    print(sqlCopy)

    ### criando tabela de apoio para o direct query do BI
    cur.execute("CREATE TABLE IF NOT EXISTS _measures (_measures int); TRUNCATE TABLE _measures; INSERT INTO _measures (_measures) VALUES (1);")
    ### tem q criar a tabela de data

    cur.execute(sqlCreate)
    cur.execute('TRUNCATE TABLE ' + tablename)
    cur.execute(sqlCopy)
    pg_conn.commit()
    cur.close()

    #sql_delete = '''DROP TABLE IF EXISTS copy_test;'''

    os.remove(os.path.join(path,tablename + ".csv"))

    return 1


copyDataToSQL(createCorrelDFrame(numDF[numFList+tgtFList+discFList]),"correlTable")
copyDataToSQL(numDF,"Numeric")
copyDataToSQL(numDFMelt,"meltedNumeric")
copyDataToSQL(dim_fDF,"dim_Features")
copyDataToSQL(dim_idDF,"dim_id")

CREATE TABLE IF NOT EXISTS correlTable (variable1 varchar(100), variable2 varchar(100), correl double precision)
COPY correlTable FROM 'D:\Projetos\competicoes\featureOps\data\tmp\correlTable.csv' DELIMITER ',' CSV; 
CREATE TABLE IF NOT EXISTS Numeric (id bigint, f_00 double precision, f_01 double precision, f_02 double precision, f_03 double precision, f_04 double precision, f_05 double precision, f_06 double precision, f_07 bigint, f_08 bigint, f_09 bigint, f_10 bigint, f_11 bigint, f_12 bigint, f_13 bigint, f_14 bigint, f_15 bigint, f_16 bigint, f_17 bigint, f_18 bigint, f_19 double precision, f_20 double precision, f_21 double precision, f_22 double precision, f_23 double precision, f_24 double precision, f_25 double precision, f_26 double precision, f_27 varchar(100), f_28 double precision, f_29 bigint, f_30 bigint, target bigint)
COPY Numeric FROM 'D:\Projetos\competicoes\featureOps\data\tmp\Numeric.csv' DELIMITER ',' CSV; 
CREATE TABLE IF NOT EXISTS meltedNumeric (id bigint, f_2

1